In [2]:
# yeni kaggle
import pydicom 
import cv2
import matplotlib.pyplot as plt
import glob
import gdcm
import pydicom
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from joblib import Parallel, delayed 
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import re
import pydicom
from skimage import exposure
import os 
import skimage 

In [21]:
df = pd.read_csv('rsna_bitirme2.csv')

SAVE_FOLDER = "C:\\Users\\alper\\verisetleri\\rsna_bitirme"
os.makedirs(SAVE_FOLDER, exist_ok=True)
def fit_image(fname, size=512):
    # 1. Read, resize
    
    # dicom = pydicom.dcmread(fname)
    # img = dicom.pixel_array
    img = skimage.io.imread(fname)
    img = (img - img.min()) / (img.max() - img.min())
    # if dicom.PhotometricInterpretation == "MONOCHROME1":
    #     img = 1 - img
    # img = cv2.resize(img, (size, size))
    
    # 2. Crop
    X = img
    # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
    X = X[5:-5, 5:-5]
    X = (X * 255).astype(np.uint8)
    
    
    # regions of non-empty pixels
    output= cv2.connectedComponentsWithStats((X > 20).astype(np.uint8)[:, :], 8, cv2.CV_32S)

    # stats.shape == (N, 5), where N is the number of regions, 5 dimensions correspond to:
    # left, top, width, height, area_size
    stats = output[2]

    # finding max area which always corresponds to the breast data. 
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h

    # cutting out the breast data
    X_fit = X[y1: y2, x1: x2]
    
    gamma2=exposure.adjust_gamma(X_fit,2.5) 
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    clahe1 = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))

    gamma2 = clahe.apply(gamma2) 
    gamma2 = clahe1.apply(gamma2) 

    img = cv2.resize(gamma2, (size, size))
    # print('ok')
    
    patient = fname.split('\\')[-2]
    image = fname.split('\\')[-1][:-4]
    image = image.split('_')[1]

    view = df[df['image_id'] == int(image)]['laterality'].values + df[df['image_id'] == int(image)]['view'].values

    if view == 'RCC' or view == "RMLO":
        img = cv2.flip(img, 1)

    os.makedirs(SAVE_FOLDER + '\\' + str(patient), exist_ok=True)

    cv2.imwrite(SAVE_FOLDER + '\\' + str(patient) + f"\\{view}.png",  img)
    

def fit_all_images(all_images):
    with ThreadPoolExecutor(4) as p:
        for i in tqdm(p.map(fit_image, all_images), total=len(all_images)):
            pass

all_images = glob.glob("C:\\Users\\alper\\OneDrive\\Masaüstü\\bitirme\\rnsa_bitirme/*/*.png")

fit_all_images(all_images)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html